In [1]:
!pip install datasets transformers mecab-python3 unidic-lite bpemb

     |████████████████████████████████| 282 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 47.4 MB 29.3 MB/s eta 0:00:01
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=acd7ebf1da57a3cdab7e78296c1d54e12624402381d194d6448b8799f4072aea
  Stored in directory: /Users/knf792/Library/Caches/pip/wheels/56/9c/4f/2c115e896b4b6c584039ca19de3581d333856782ef108cdc5c
Successfully built unidic-lite


#Preprocessing 1.1

In [2]:
from datasets import load_dataset, Dataset
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

Using custom data configuration copenlu___answerable_tydiqa-42333912ea665dd0


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/71.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.49M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /Users/knf792/.cache/huggingface/datasets/parquet/copenlu___answerable_tydiqa-42333912ea665dd0/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train_set

Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
    num_rows: 116067
})

In [8]:
validation_set

Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
    num_rows: 13325
})

Remove all languages from the data set (besides english, finnish and japanese)

In [3]:
import pandas as pd
df_train = pd.DataFrame(train_set)
df_train = df_train.dropna()

df_validation = pd.DataFrame(validation_set)
df_validation = df_validation.dropna()

In [4]:
# filter for required languages
eng_train_set = df_train.loc[df_train.language == "english"]
fin_train_set = df_train.loc[df_train.language == "finnish"]
jap_train_set = df_train.loc[df_train.language == "japanese"]

eng_validation_set = df_validation.loc[df_validation.language == "english"]
fin_validation_set = df_validation.loc[df_validation.language == "finnish"]
jap_validation_set = df_validation.loc[df_validation.language == "japanese"]

# Language modelling 3

### Transformers Finnish (Marianne)

In [5]:
import transformers
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import math


In [6]:
#fin_model = transformers.AutoModelForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1") 
#fin_model = transformers.BertForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1") 
fin_model = AutoModelForCausalLM.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1") 

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
import torch

In [7]:
#fin_model.eval()
#if torch.cuda.is_available():
#    fin_model = fin_model.cuda()
#fin_tokenizer = transformers.BertTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
fin_tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/414k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/796k [00:00<?, ?B/s]

In [9]:
fin_train_trans = fin_train_set[["document_plaintext", "annotations"]]
fin_train_trans["annotations"] = fin_train_trans["annotations"].apply(lambda x: 1 if x["answer_start"][0] != -1 else 0)
fin_train_trans = fin_train_trans.rename(columns={"document_plaintext":"text", "annotations": "label"})
fin_train_trans = fin_train_trans.sample(frac=1)

fin_valid_trans = fin_validation_set[["document_plaintext", "annotations"]]
fin_valid_trans["annotations"] = fin_valid_trans["annotations"].apply(lambda x: 1 if x["answer_start"][0] != -1 else 0)
fin_valid_trans = fin_valid_trans.rename(columns={"document_plaintext":"text", "annotations": "label"})
fin_valid_trans = fin_valid_trans.sample(frac=1)

fin_totdat_trans = pd.concat([fin_train_trans, fin_valid_trans], axis=0)
fin_totdat_trans

<ipython-input-9-2ec4d3ca23cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_train_trans["annotations"] = fin_train_trans["annotations"].apply(lambda x: 1 if x["answer_start"][0] != -1 else 0)
<ipython-input-9-2ec4d3ca23cc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_valid_trans["annotations"] = fin_valid_trans["annotations"].apply(lambda x: 1 if x["answer_start"][0] != -1 else 0)


,text,label
29367,Louis Pierre Althusser (16. lokakuuta 1918 – 2...,1
18446,Suomi antoi kolmantena maailmassa ja ensimmäis...,1
20101,Seattle on Yhdysvaltojen luoteisosan suurin ka...,1
19837,Eero Olavi Mantere (25. lokakuuta 1904 Helsink...,1
12962,"Järven pituus on 8,4 kilometriä, leveys 4,6 ki...",1
...,...,...
12301,13-vuotiaana hän muutti veljensä luokse Honolu...,0
3487,\nTalvisota oli 30. marraskuuta 1939 – 13. maa...,1
4038,1900-luvun alussa väkiluku oli noussut jo lähe...,1
11734,Kekkonen palasi lopullisesti vallan sisärenkaa...,0


In [10]:
import datasets
fin_totdat_trans = datasets.arrow_dataset.Dataset.from_pandas(fin_totdat_trans)
fin_totdat_trans = fin_totdat_trans.train_test_split(test_size=0.10)
fin_totdat_trans

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 13848
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1539
    })
})

In [11]:
#block_size remained undefined for some reason idk
import numpy as np

def fin_tokenize_function(examples):
    return fin_tokenizer(examples["text"])

block_size = 256
def fin_group_texts(examples):
    # Concatenate all texts.
    keys = ['attention_mask', 'input_ids']
    #concatenated_examples = {k: sum(examples[k], []) for k in keys}
    concatenated_examples = {}

    att_list = []
    for att in examples["attention_mask"]:
      if isinstance(att, int):
        att_list.append([att])
      else:
        att_list.append(att)
    
    concatenated_examples['attention_mask'] = list(np.concatenate(att_list).flat)

    att_list = []
    for att in examples['input_ids']:
      if isinstance(att, int):
        att_list.append([att])
      else:
        att_list.append(att)

    concatenated_examples['input_ids'] = list(np.concatenate(att_list).flat)

    #concatenated_examples = {k: list(np.concatenate(examples[k]).flat) for k in keys}
    # list(np.concatenate(original_list). flat)
    total_length = len(concatenated_examples[list(keys)[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # this is needed as the used dataset is a subclass of ClassificationDataset, which requires label as a field...
    result["label"] = result["input_ids"].copy()
    result["labels"] = result["input_ids"].copy()
    return result


In [12]:
fin_totdat_trans = fin_totdat_trans.map(fin_tokenize_function, remove_columns=["text", '__index_level_0__'])

  0%|          | 0/13848 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1986 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/1539 [00:00<?, ?ex/s]

In [13]:
fin_totdat_trans = fin_totdat_trans.remove_columns(['token_type_ids'])
fin_totdat_trans

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 13848
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1539
    })
})

In [14]:
fin_totdat_trans = fin_totdat_trans.map(fin_group_texts, batched=True, batch_size=1000, num_proc=4,)
fin_totdat_trans = fin_totdat_trans.remove_columns(['label'])

In [15]:
#fin_totdat_trans = fin_totdat_trans.remove_columns(['label', 'labels'])
fin_totdat_trans

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 5472
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 609
    })
})

In [16]:
fin_totdat_trans["test"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [17]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

fin_training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    max_steps=300,
    #label_names='label'
)

In [18]:
fin_trainer = Trainer(
    model=fin_model,
    args=fin_training_args,
    train_dataset=fin_totdat_trans["train"],
    eval_dataset=fin_totdat_trans["test"]
)

fin_trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/Users/knf792/miniconda3/envs/black_white/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5472
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 300
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: copenlu (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [108]:
eval_results['eval_loss']

0.021662350744009018

In [100]:
eval_results = fin_trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 621
  Batch size = 8


Perplexity: 1.02


In [101]:
fin_model.save_pretrained("fin_bert_tydiqa")

Configuration saved in fin_bert_tydiqa/config.json
Model weights saved in fin_bert_tydiqa/pytorch_model.bin


In [102]:
fin_model = fin_model.to('cpu')

In [103]:
# encode context the generation is conditioned on
input_ids = fin_tokenizer.encode('Kun Lisa lähtee kotoaan', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = fin_model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(fin_tokenizer.decode(greedy_output[0], skip_special_tokens=True))

# end of sequence tokens?

Output:
----------------------------------------------------------------------------------------------------
Kun Lisa lähtee kotoaan Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun Kun


In [104]:
# encode context the generation is conditioned on
input_ids = fin_tokenizer.encode('Tänään olen menossa rannalle, haluaisin mennä sinne', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = fin_model.generate(input_ids, max_length=25)

print("Output:\n" + 100 * '-')
print(fin_tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Tänään olen menossa rannalle, haluaisin mennä sinne...............


In [107]:
# encode context the generation is conditioned on
input_ids = fin_tokenizer.encode('Miksi tekee', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = fin_model.generate(input_ids, max_length=25)

print("Output:\n" + 100 * '-')
print(fin_tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Miksi tekee Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän Hän
